# **Modelo de Previsão de Preços de Imóveis em São Paulo**

Importação das principais bibliotecas e da base de dados

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("argonalyst/sao-paulo-real-estate-sale-rent-april-2019")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import geopandas as gpd
import contextily as ctx

### 1 - Análise Exploratória da Base de Dados e Tratamento de Dados

In [ ]:
# Caminho do arquivo com barras duplas (ou use raw string com prefixo 'r')
caminho_csv = r'...\kagglehub\datasets\argonalyst\sao-paulo-real-estate-sale-rent-april-2019\versions\1\sao-paulo-properties-april-2019.csv'

# Lendo o arquivo
df = pd.read_csv(caminho_csv)

In [ ]:
df.head()

In [ ]:
# Visualização da distribuição por apartamentos por quartos
px.scatter(df, x='Rooms', y='Price')

In [ ]:
df['Negotiation Type'].unique()

In [ ]:
df['District'].unique()

In [ ]:
mapeamento_regioes = {
    # Zona Leste
    'Artur Alvim/São Paulo': 'Zona Leste',
    'Cangaíba/São Paulo': 'Zona Leste',
    'Carrão/São Paulo': 'Zona Leste',
    'Cidade Líder/São Paulo': 'Zona Leste',
    'Cidade Tiradentes/São Paulo': 'Zona Leste',
    'Ermelino Matarazzo/São Paulo': 'Zona Leste',
    'Itaim Paulista/São Paulo': 'Zona Leste',
    'Itaquera/São Paulo': 'Zona Leste',
    'Jardim Helena/São Paulo': 'Zona Leste',
    'José Bonifácio/São Paulo': 'Zona Leste',
    'Lajeado/São Paulo': 'Zona Leste',
    'Parque do Carmo/São Paulo': 'Zona Leste',
    'Ponte Rasa/São Paulo': 'Zona Leste',
    'São Mateus/São Paulo': 'Zona Leste',
    'São Miguel/São Paulo': 'Zona Leste',
    'São Rafael/São Paulo': 'Zona Leste',
    'Vila Curuçá/São Paulo': 'Zona Leste',
    'Vila Formosa/São Paulo': 'Zona Leste',
    'Aricanduva/São Paulo': 'Zona Leste',
    'Guaianazes/São Paulo': 'Zona Leste',
    'Penha/São Paulo': 'Zona Leste',
    'Tatuapé/São Paulo': 'Zona Leste',
    'Vila Matilde/São Paulo': 'Zona Leste',
    'Mooca/São Paulo': 'Zona Leste',
    'Belém/São Paulo': 'Zona Leste',
    'Água Rasa/São Paulo': 'Zona Leste',
    'Sapopemba/São Paulo': 'Zona Leste',
    'São Lucas/São Paulo': 'Zona Leste',
    'Vila Jacuí/São Paulo': 'Zona Leste',
    'Vila Prudente/São Paulo': 'Zona Leste',
    'Iguatemi/São Paulo': 'Zona Leste',
    
    # Zona Sul
    'Campo Belo/São Paulo': 'Zona Sul',
    'Campo Grande/São Paulo': 'Zona Sul',
    'Campo Limpo/São Paulo': 'Zona Sul',
    'Capão Redondo/São Paulo': 'Zona Sul',
    'Cidade Ademar/São Paulo': 'Zona Sul',
    'Cidade Dutra/São Paulo': 'Zona Sul',
    'Grajaú/São Paulo': 'Zona Sul',
    'Ipiranga/São Paulo': 'Zona Sul',
    'Itaim Bibi/São Paulo': 'Zona Sul',
    'Jabaquara/São Paulo': 'Zona Sul',
    'Jardim Ângela/São Paulo': 'Zona Sul',
    'Jardim Paulista/São Paulo': 'Zona Sul',
    'Moema/São Paulo': 'Zona Sul',
    'Morumbi/São Paulo': 'Zona Sul',
    'Sacomã/São Paulo': 'Zona Sul',
    'Santo Amaro/São Paulo': 'Zona Sul',
    'Saúde/São Paulo': 'Zona Sul',
    'Socorro/São Paulo': 'Zona Sul',
    'Vila Andrade/São Paulo': 'Zona Sul',
    'Vila Mariana/São Paulo': 'Zona Sul',
    'Vila Sônia/São Paulo': 'Zona Sul',
    'Brooklin/São Paulo': 'Zona Sul',
    'Vila Olimpia/São Paulo': 'Zona Sul',
    'Pedreira/São Paulo': 'Zona Sul',
    'Cursino/São Paulo': 'Zona Sul',
    'Cambuci/São Paulo': 'Zona Sul',
    'Jardim São Luis/São Paulo': 'Zona Sul',
    
    # Zona Oeste
    'Alto de Pinheiros/São Paulo': 'Zona Oeste',
    'Butantã/São Paulo': 'Zona Oeste',
    'Jaguaré/São Paulo': 'Zona Oeste',
    'Lapa/São Paulo': 'Zona Oeste',
    'Perdizes/São Paulo': 'Zona Oeste',
    'Pinheiros/São Paulo': 'Zona Oeste',
    'Vila Leopoldina/São Paulo': 'Zona Oeste',
    'Vila Madalena/São Paulo': 'Zona Oeste',
    'Raposo Tavares/São Paulo': 'Zona Oeste',
    'Rio Pequeno/São Paulo': 'Zona Oeste',
    'Pirituba/São Paulo': 'Zona Oeste',
    'Perus/São Paulo': 'Zona Oeste',
    'Anhanguera/São Paulo': 'Zona Oeste',
    'Jaraguá/São Paulo': 'Zona Oeste',
    'São Domingos/São Paulo': 'Zona Oeste',
    
    # Zona Norte
    'Casa Verde/São Paulo': 'Zona Norte',
    'Santana/São Paulo': 'Zona Norte',
    'Tucuruvi/São Paulo': 'Zona Norte',
    'Mandaqui/São Paulo': 'Zona Norte',
    'Tremembé/São Paulo': 'Zona Norte',
    'Vila Guilherme/São Paulo': 'Zona Norte',
    'Vila Maria/São Paulo': 'Zona Norte',
    'Jaçanã/São Paulo': 'Zona Norte',
    'Brasilândia/São Paulo': 'Zona Norte',
    'Freguesia do Ó/São Paulo': 'Zona Norte',
    'Limão/São Paulo': 'Zona Norte',
    'Cachoeirinha/São Paulo': 'Zona Norte',
    'Medeiros/São Paulo': 'Zona Norte',
    
    # Centro (se necessário)
    'Barra Funda/São Paulo': 'Centro',
    'Bela Vista/São Paulo': 'Centro',
    'Bom Retiro/São Paulo': 'Centro',
    'Brás/São Paulo': 'Centro',
    'Consolação/São Paulo': 'Centro',
    'Liberdade/São Paulo': 'Centro',
    'República/São Paulo': 'Centro',
    'Santa Cecília/São Paulo': 'Centro',
    'Sé/São Paulo': 'Centro',
    'Pari/São Paulo': 'Centro'
}


In [ ]:
df['Regiao'] = df['District'].map(mapeamento_regioes)

In [ ]:
df['Regiao'] 

In [ ]:
df.describe()

In [ ]:
px.density_mapbox(df, lat="Latitude", lon="Longitude", radius=10, zoom=10, mapbox_style="open-street-map")

In [ ]:
df[["bairro", "municipio"]] = df["District"].str.split("/", expand=True)
df = df[df["New"] == 0]
df = df[df['Negotiation Type'] == 'sale']
df.drop(['New', 'Negotiation Type','Property Type'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# Histograma interativo com Plotly
fig = px.histogram(
    df,
    x="Price",                  # Coluna para o eixo X
    nbins=200,                    # Número de intervalos
    title="Histograma de Valores (São Paulo/SP)",  # Título
    labels={"valor": "Valor"},   # Rótulo do eixo X
    opacity=0.7,                # Transparência das barras
    color_discrete_sequence=["#1f77b4"],  # Cor das barras (azul)
    template="plotly_white"     # Estilo do gráfico
)

# Personalizar layout
fig.update_layout(
    bargap=0.1,                 # Espaço entre barras
    xaxis_title="Valor",        # Título do eixo X
    yaxis_title="Frequência",   # Título do eixo Y
    hovermode="x",              # Mostrar informações ao passar o mouse
    showlegend=False            # Ocultar legenda (não necessária para histogramas simples)
)

# Exibir o gráfico
fig.show()

In [ ]:
# 1. Subplots para histogramas (Price e Size)
fig = make_subplots(
    rows=1, cols=2,  # 1 linha, 2 colunas
    subplot_titles=("Distribuição de Preços", "Distribuição de Metragem"),
    horizontal_spacing=0.15
)

# Histograma de Preços
fig.add_trace(
    go.Histogram(x=df["Price"], nbinsx=100, name="Preço", marker_color="#1f77b4"),
    row=1, col=1
)

# Histograma do tamanho do apartamento
fig.add_trace(
    go.Histogram(x=df["Size"], nbinsx=100, name="Metragem", marker_color="#ff7f0e"),
    row=1, col=2
)

# Atualizar layout dos subplots
fig.update_layout(
    title_text="Análise de Imóveis: Preço e Metragem",
    showlegend=False,
    height=400
)

# 2. Gráfico agrupado por bairro (usando Plotly Express)
fig_bairro = px.bar(
    df.groupby("bairro", as_index=False)["Price"].mean(),
    x="bairro",
    y="Price",
    title="Preço Médio por Bairro",
    color="bairro",
    labels={"Price": "Preço Médio", "Neighborhood": "Bairro"}
)
fig_bairro.update_layout(height=500, showlegend=False)

# Exibir os gráficos
fig.show()
fig_bairro.show()


### 1 - Pré-processamento dos Dados Consolidados

In [ ]:
df.head(100)

In [ ]:
df.drop(['District'], axis=1, inplace=True)

In [ ]:
df.drop(['municipio'], axis=1, inplace=True)

In [ ]:
df.drop(['Condo'], axis=1, inplace=True)

In [ ]:
df.drop(['Furnished'], axis=1, inplace=True)

In [ ]:
df.drop(['Elevator'], axis=1, inplace=True)

In [ ]:
df.drop(['Latitude'], axis=1, inplace=True)

In [ ]:
df.drop(['Longitude'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
#Selecionar features relevantes
features = ["Size", "Rooms", "Toilets", "Suites", "Parking"]
target = "Price"

# Tratamento de valores ausentes
df.dropna(subset=['Price', 'Size'], inplace=True)

### 2. Engenharia de Features com Foco em Localização

In [ ]:
# Criar preço por m²
df['preco_m2'] = df['Price'] / df['Size']

# Calcular médias por região para features de localização
df['media_preco_bairro'] = df.groupby('bairro')['preco_m2'].transform('mean')
df['media_preco_regiao'] = df.groupby('Regiao')['preco_m2'].transform('mean')

In [ ]:
df.head()

In [ ]:
# Codificar hierarquia geográfica (Distrito > Região)
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[sorted(df['Regiao'].unique()), 
                         sorted(df['bairro'].unique())])
df[['REGIAO_ENC', 'DISTRITO_ENC']] = encoder.fit_transform(df[['Regiao', 'bairro']])

# Features finais para o modelo
model_features = [
    'Size', 'Rooms', 'Parking', 'Toilets', 'Suites', 'Regiao', 'bairro'
]

### 3. Pipeline de Modelagem com XGBoost

In [ ]:

# Divisão dos dados
X = df[model_features]
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Size']),
        ('cat', OneHotEncoder(), ['Regiao', 'bairro'])
    ],
    remainder='passthrough'
)

In [ ]:
preprocessor

In [ ]:
# Modelo XGBoost com otimização de hiperparâmetros
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.8,
        random_state=42
    ))
])

In [ ]:
# Treinamento
model.fit(X_train, y_train)

In [ ]:
df.head()

In [ ]:
# Avaliação
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"mae: {mae:.2f}")
print(f"R²: {r2:.4f}")

In [ ]:
# Avaliação
y_pred = model.predict(X_test)
mae = np.sqrt(mean_absolute_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")

### 4. Análise da Importância de Features

In [ ]:
# Extrair importância das features
feature_importances = model.named_steps['regressor'].feature_importances_

In [ ]:
# Obter nomes das features após o pré-processamento
cat_features = model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(['Regiao', 'bairro'])
all_features = ['Size'] + list(cat_features) + model_features[3:]

In [ ]:
# Criar DataFrame com importâncias
importance_df = pd.DataFrame({
    'Feature': all_features,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

In [ ]:
# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
plt.title('Top 20 Features Mais Importantes para Previsão de Preços')
plt.show()

In [ ]:
import joblib

# Salvar o modelo
joblib.dump(model, r'C:\Users\luuka\Downloads\modelo_precos_sp.pkl')

In [ ]:
import joblib
import pandas as pd

def prever_preco(Size, Rooms, Parking, bairro, Suites, Toilets, Regiao):
    """
    Função para prever preços de imóveis com todos os parâmetros necessários
    
    Parâmetros obrigatórios:
    - Size: Área construída em m² (float)
    - Rooms: Número de quartos (int)
    - Parking: Vagas de garagem (int)
    - bairro: Nome do bairro (str)
    - Suites: Número de suítes (int)
    - Toilets: Número de banheiros (int)
    
    Parâmetro opcional:
    - Regiao: Região da cidade (str) - pode ser None
    """
    try:
        # Carregar modelo
        model = joblib.load(r'C:\Users\luuka\Downloads\modelo_precos_sp.pkl')
        
        # Criar DataFrame com todos os parâmetros
        input_data = pd.DataFrame({
            'Size': [Size],
            'Rooms': [Rooms],
            'Parking': [Parking],
            'bairro': [bairro],
            'Suites': [Suites],
            'Toilets': [Toilets],
            'Regiao': [Regiao]
        })
        
        
        # Calcular média do bairro
        input_data['MEDIA_PRECO_DISTRITO'] = df[df['bairro'] == bairro]['preco_m2'].mean()
        
        # Fazer previsão
        return model.predict(input_data)[0]
    
    except Exception as e:
        print(f"Erro na previsão: {str(e)}")
        return None

In [ ]:
prever_preco

In [ ]:
prever_preco(59, 2, 1, 'Ipiranga', 1, 2, 'Zona Sul')

In [ ]:
mapeamento_regioes = {
    # Zona Leste
    'Artur Alvim': 'Zona Leste',
    'Cangaíba': 'Zona Leste',
    'Carrão': 'Zona Leste',
    'Cidade Líder': 'Zona Leste',
    'Cidade Tiradentes': 'Zona Leste',
    'Ermelino Matarazzo': 'Zona Leste',
    'Itaim Paulista': 'Zona Leste',
    'Itaquera': 'Zona Leste',
    'Jardim Helena': 'Zona Leste',
    'José Bonifácio': 'Zona Leste',
    'Lajeado': 'Zona Leste',
    'Parque do Carmo': 'Zona Leste',
    'Ponte Rasa': 'Zona Leste',
    'São Mateus': 'Zona Leste',
    'São Miguel': 'Zona Leste',
    'São Rafael': 'Zona Leste',
    'Vila Curuçá': 'Zona Leste',
    'Vila Formosa': 'Zona Leste',
    'Aricanduva': 'Zona Leste',
    'Guaianazes': 'Zona Leste',
    'Penha': 'Zona Leste',
    'Tatuapé': 'Zona Leste',
    'Vila Matilde': 'Zona Leste',
    'Mooca': 'Zona Leste',
    'Belém': 'Zona Leste',
    'Água Rasa': 'Zona Leste',
    'Sapopemba': 'Zona Leste',
    'São Lucas': 'Zona Leste',
    'Vila Jacuí': 'Zona Leste',
    'Vila Prudente': 'Zona Leste',
    'Iguatemi': 'Zona Leste',
    
    # Zona Sul
    'Campo Belo': 'Zona Sul',
    'Campo Grande': 'Zona Sul',
    'Campo Limpo': 'Zona Sul',
    'Capão Redondo': 'Zona Sul',
    'Cidade Ademar': 'Zona Sul',
    'Cidade Dutra': 'Zona Sul',
    'Grajaú': 'Zona Sul',
    'Ipiranga': 'Zona Sul',
    'Itaim Bibi': 'Zona Sul',
    'Jabaquara': 'Zona Sul',
    'Jardim Ângela': 'Zona Sul',
    'Jardim Paulista': 'Zona Sul',
    'Moema': 'Zona Sul',
    'Morumbi': 'Zona Sul',
    'Sacomã': 'Zona Sul',
    'Santo Amaro': 'Zona Sul',
    'Saúde': 'Zona Sul',
    'Socorro': 'Zona Sul',
    'Vila Andrade': 'Zona Sul',
    'Vila Mariana': 'Zona Sul',
    'Vila Sônia': 'Zona Sul',
    'Brooklin': 'Zona Sul',
    'Vila Olimpia': 'Zona Sul',
    'Pedreira': 'Zona Sul',
    'Cursino': 'Zona Sul',
    'Cambuci': 'Zona Sul',
    'Jardim São Luis': 'Zona Sul',
    
    # Zona Oeste
    'Alto de Pinheiros': 'Zona Oeste',
    'Butantã': 'Zona Oeste',
    'Jaguaré': 'Zona Oeste',
    'Lapa': 'Zona Oeste',
    'Perdizes': 'Zona Oeste',
    'Pinheiros': 'Zona Oeste',
    'Vila Leopoldina': 'Zona Oeste',
    'Vila Madalena': 'Zona Oeste',
    'Raposo Tavares': 'Zona Oeste',
    'Rio Pequeno': 'Zona Oeste',
    'Pirituba': 'Zona Oeste',
    'Perus': 'Zona Oeste',
    'Anhanguera': 'Zona Oeste',
    'Jaraguá': 'Zona Oeste',
    'São Domingos': 'Zona Oeste',
    
    # Zona Norte
    'Casa Verde': 'Zona Norte',
    'Santana': 'Zona Norte',
    'Tucuruvi': 'Zona Norte',
    'Mandaqui': 'Zona Norte',
    'Tremembé': 'Zona Norte',
    'Vila Guilherme': 'Zona Norte',
    'Vila Maria': 'Zona Norte',
    'Jaçanã': 'Zona Norte',
    'Brasilândia': 'Zona Norte',
    'Freguesia do Ó': 'Zona Norte',
    'Limão': 'Zona Norte',
    'Cachoeirinha': 'Zona Norte',
    'Medeiros': 'Zona Norte',
    
    # Centro
    'Barra Funda': 'Centro',
    'Bela Vista': 'Centro',
    'Bom Retiro': 'Centro',
    'Brás': 'Centro',
    'Consolação': 'Centro',
    'Liberdade': 'Centro',
    'República': 'Centro',
    'Santa Cecília': 'Centro',
    'Sé': 'Centro',
    'Pari': 'Centro'
}

def previsao_interativa():
    """Função interativa que determina a região automaticamente pelo bairro"""
    
    print("\n=== PREVISÃO DE PREÇOS DE IMÓVEIS EM SÃO PAULO ===")
    print("Por favor, informe os dados solicitados (digite 'sair' a qualquer momento para cancelar):\n")
    
    try:
        # Coletar dados do imóvel
        print("▌ DADOS DO IMÓVEL ▌")
        Size = input("• Área construída (m²): ")
        if Size.lower() == 'sair': return
        Size = float(Size)
        
        Rooms = input("• Número de quartos: ")
        if Rooms.lower() == 'sair': return
        Rooms = int(Rooms)
        
        Suites = input("• Número de suítes: ")
        if Suites.lower() == 'sair': return
        Suites = int(Suites)
        
        Toilets = input("• Número de banheiros: ")
        if Toilets.lower() == 'sair': return
        Toilets = int(Toilets)
        
        Parking = input("• Vagas de garagem: ")
        if Parking.lower() == 'sair': return
        Parking = int(Parking)
        
        # Coletar localização com tratamento especial
        print("\n▌ LOCALIZAÇÃO ▌")
        while True:
            bairro = input("• Bairro (ex: 'Pinheiros', 'Moema'): ").strip().title()
            if bairro.lower() == 'sair': return
            
            # Verificar se o bairro está no mapeamento
            Regiao = mapeamento_regioes.get(bairro, None)
            
            if Regiao is not None:
                break
            else:
                print(f"Bairro '{bairro}' não encontrado. Por favor, digite novamente.")
                print("Dica: Digite apenas o nome do bairro (ex: 'Tatuapé' em vez de 'Tatuapé/São Paulo')")
        
        print(f"\nℹ️ Região determinada automaticamente: {Regiao}")
        
        # Fazer previsão
        preco = prever_preco(
            Size=Size,
            Rooms=Rooms,
            Parking=Parking,
            bairro=bairro,
            Suites=Suites,
            Toilets=Toilets,
            Regiao=Regiao
        )
        
        # Mostrar resultado completo
        if preco is not None:
            print("\n" + "="*60)
            print("📋 RESUMO DOS DADOS INFORMADOS")
            print("-"*60)
            print(f"▪ Área construída: {Size} m²")
            print(f"▪ Quartos: {Rooms} (Suítes: {Suites})")
            print(f"▪ Banheiros: {Toilets}")
            print(f"▪ Vagas: {Parking}")
            print(f"▪ Localização: {bairro} - {Regiao}")
            
            print("\n💲 RESULTADO DA PREVISÃO")
            print("-"*60)
            print(f"Valor estimado: R$ {preco:,.2f}")
            print(f"Faixa sugerida: R$ {preco*0.85:,.2f} a R$ {preco*1.15:,.2f}")
            print("="*60)
        else:
            print("\nNão foi possível calcular a previsão.")
            
    except ValueError:
        print("\nErro: Por favor, digite valores numéricos nos campos solicitados.")
    except Exception as e:
        print(f"\nOcorreu um erro: {str(e)}")

# Instruções para uso:
print("\nINSTRUÇÕES:")
print("1. Digite apenas o nome do bairro (sem '/São Paulo')")
print("2. O sistema identificará automaticamente a região")
print("3. Digite 'sair' a qualquer momento para cancelar\n")

# Exemplo de uso:
previsao_interativa()

In [ ]:
def previsao_interativa():
    """Função interativa que determina a região automaticamente pelo bairro"""
    
    print("\n=== PREVISÃO DE PREÇOS DE IMÓVEIS EM SÃO PAULO ===")
    print("Por favor, informe os dados solicitados (digite 'sair' a qualquer momento para cancelar):\n")
    
    try:
        # Coletar dados do imóvel
        print("▌ DADOS DO IMÓVEL ▌")
        Size = input("• Área construída (m²): ")
        if Size.lower() == 'sair': return
        Size = float(Size)
        if Size <= 10:
            print("❌ Valor inválido. A área mínima deve ser maior que 10m². Digite novamente.")
            return
        Rooms = input("• Número de quartos: ")
        if Rooms.lower() == 'sair': return
        Rooms = int(Rooms)
        if Rooms < 1:
            print("❌ Valor inválido. O valor mínimo é 1.")
            return
        Suites = input("• Número de suítes: ")
        if Suites.lower() == 'sair': return
        Suites = int(Suites)
        
        Toilets = input("• Número de banheiros: ")
        if Toilets.lower() == 'sair': return
        Toilets = int(Toilets)
        if Toilets < 1:
            print("❌ Valor inválido. O valor mínimo é 1.")
            return
        Parking = input("• Vagas de garagem: ")
        if Parking.lower() == 'sair': return
        Parking = int(Parking)
        
        # Coletar localização com tratamento especial
        print("\n▌ LOCALIZAÇÃO ▌")
        while True:
            bairro = input("• Bairro (ex: 'Pinheiros', 'Moema'): ").strip().title()
            if bairro.lower() == 'sair': return
            
            # Verificar se o bairro está no mapeamento
            Regiao = mapeamento_regioes.get(bairro, None)
            
            if Regiao is not None:
                break
            else:
                print(f"Bairro '{bairro}' não encontrado. Por favor, digite novamente.")
                print("Dica: Digite apenas o nome do bairro (ex: 'Tatuapé' em vez de 'Tatuapé/São Paulo')")
        
        print(f"\nℹ️ Região determinada automaticamente: {Regiao}")
        
        # Fazer previsão
        preco = prever_preco(
            Size=Size,
            Rooms=Rooms,
            Parking=Parking,
            bairro=bairro,
            Suites=Suites,
            Toilets=Toilets,
            Regiao=Regiao
        )
        
        # Mostrar resultado completo
        if preco is not None:
            print("\n" + "="*60)
            print("📋 RESUMO DOS DADOS INFORMADOS")
            print("-"*60)
            print(f"▪ Área construída: {Size} m²")
            print(f"▪ Quartos: {Rooms} (Suítes: {Suites})")
            print(f"▪ Banheiros: {Toilets}")
            print(f"▪ Vagas: {Parking}")
            print(f"▪ Localização: {bairro} - {Regiao}")
            
            print("\n💲 RESULTADO DA PREVISÃO")
            print("-"*60)
            print(f"Valor estimado: R$ {preco:,.2f}")
            print(f"Faixa sugerida: R$ {preco*0.85:,.2f} a R$ {preco*1.15:,.2f}")
            print("="*60)
        else:
            print("\nNão foi possível calcular a previsão.")
            
    except ValueError:
        print("\nErro: Por favor, digite valores numéricos nos campos solicitados.")
    except Exception as e:
        print(f"\nOcorreu um erro: {str(e)}")

# Instruções para uso:
print("\nINSTRUÇÕES:")
print("1. Digite apenas o nome do bairro (sem '/São Paulo')")
print("2. O sistema identificará automaticamente a região")
print("3. Digite 'sair' a qualquer momento para cancelar\n")

# Exemplo de uso:
previsao_interativa()